In [1]:
from SOM import SOM
import sys

sys.path.append("../../")
from utils import load_csv, dataloader, load_save_models
import torch
from torchvision.transforms import v2
import preprocessing.tiff_handling as tiff_handling
import plotly.express as px
from stats import evaluate_perf_utils
import numpy as np

## load in the data

In [2]:
input_dim = 100
map_dim = 80

In [3]:
model = SOM.load_model(SOM(input_dim=input_dim,map_dim=map_dim),'./hpc_SOM_model.pkl')
model.som

Model loaded from ./hpc_SOM_model.pkl.


## load in data

In [4]:
set = load_csv.load_pandas()
train, val,test = load_csv.split_data(set)

print(f'outlier test validation: {any(val["classification"]!=1)}')

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/SOM/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


outlier test validation: False


In [5]:
transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        # v2.Lambda(
        #     lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        # ),
    ]
)

val_data = np.array(
    [
        tiff_handling.flatten_image(transform(tiff_handling.handle_tiff(el)))
        for el in val["file_names"].tolist()
    ]
)

test["image"] = test["file_names"].apply(lambda x: tiff_handling.flatten_image(transform(tiff_handling.handle_tiff(x))))

In [6]:
model.set_threshold(val_data)

Threshold set at 2.8702 (Percentile: 95)


np.float64(2.870176573873579)

In [ ]:
output = model.predict(test)

In [ ]:
truth = [1 if value == 1 else 0 for value in output['classification']]

perf_metrics = evaluate_perf_utils.get_perf_metrics(truth, output['prediction'])
tn = perf_metrics['tn']
tp = perf_metrics['tp']
fn = perf_metrics['fn']
fp = perf_metrics['fp']
precision = perf_metrics['precision']
recall = perf_metrics['recall']
f1 = perf_metrics['f1']

perf_metrics

In [ ]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(f'accuracy: {accuracy}')